# Causal Flight Cancellation Prediction

**Project Goal**: Predict flight cancellations and attribute them to specific causes using causal inference.

**Timeline**: 4-6 weeks | **Focus**: Causation, not just correlation


In [ ]:
## Phase 1: Setup & Imports


In [2]:
# Standard libraries
import os
import warnings
warnings.filterwarnings('ignore')

# Data manipulation
import pandas as pd
import numpy as np

# Data collection
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
import xgboost as xgb
import lightgbm as lgb

# Deep Learning
import torch
import torch.nn as nn

# Causal Inference
import dowhy
from dowhy import CausalModel

# Attribution
import shap

# Utilities
from tqdm import tqdm
import pickle

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All imports successful!")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 PyTorch version: {torch.__version__}")
print(f"📦 DoWhy version: {dowhy.__version__}")


✅ All imports successful!
📦 Pandas version: 2.3.3
📦 PyTorch version: 2.9.1
📦 DoWhy version: 0.8


## Phase 2: Data Collection

**Goal**: Collect flight data (BTS), weather data (NOAA), and metadata


In [3]:
# ============================================================================
# DATA COLLECTION: Flight Cancellation Prediction
# ============================================================================
# Sources:
# 1. BTS (Bureau of Transportation Statistics) - Flight data
# 2. NOAA - Weather data
# 3. OpenFlights - Airport/Airline metadata
# ============================================================================

# Create data directories
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
os.makedirs("data/external", exist_ok=True)


print("📁 Data directories created")


📁 Data directories created


In [53]:
def load_bts_data_multiple(filepaths, output_path=None):
    """
    Load and merge multiple BTS flight data files.
    
    Args:
        filepaths: List of file paths or single file path
        output_path: Optional path to save merged CSV
    
    Returns:
        Combined DataFrame
    """
    # Convert single filepath to list
    if isinstance(filepaths, str):
        filepaths = [filepaths]
    
    # Key columns for cancellation analysis
    key_cols = [
        'FL_DATE', 'OP_CARRIER', 'OP_CARRIER_FL_NUM', 
        'ORIGIN', 'DEST', 'CANCELLED', 'CANCELLATION_CODE',
        'CRS_DEP_TIME', 'DEP_TIME', 'CRS_ARR_TIME', 'ARR_TIME',
        'DAY_OF_WEEK', 'MONTH',
        'WEATHER_DELAY', 'CARRIER_DELAY', 'NAS_DELAY', 
        'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
        'DISTANCE', 'DIVERTED'
    ]
    
    # Load and combine all files
    dfs = []
    for filepath in filepaths:
        df = pd.read_csv(filepath, low_memory=False)
        
        # Select available columns
        available_cols = [col for col in key_cols if col in df.columns]
        df = df[available_cols].copy()
        
        # Convert FL_DATE to datetime
        if 'FL_DATE' in df.columns:
            df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
        
        dfs.append(df)
        print(f"✅ Loaded {len(df):,} flights from {filepath.split('/')[-1]}")
    
    # Combine all dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    
    # Sort by date
    if 'FL_DATE' in combined_df.columns:
        combined_df = combined_df.sort_values('FL_DATE').reset_index(drop=True)
    
    # Save merged file if output path provided
    if output_path:
        combined_df.to_csv(output_path, index=False)
        print(f"\n💾 Saved merged data to: {output_path}")
    
    # Summary statistics
    print(f"\n{'='*60}")
    print(f"COMBINED DATA SUMMARY")
    print(f"{'='*60}")
    print(f"Total flights: {len(combined_df):,}")
    print(f"Date range: {combined_df['FL_DATE'].min()} to {combined_df['FL_DATE'].max()}")
    print(f"Total cancellations: {combined_df['CANCELLED'].sum():,} ({combined_df['CANCELLED'].mean()*100:.2f}%)")
    
    # Cancellation by month
    if 'MONTH' in combined_df.columns:
        print(f"\nCancellations by month:")
        month_cancel = combined_df.groupby('MONTH')['CANCELLED'].agg(['sum', 'mean', 'count'])
        month_cancel.columns = ['Cancellations', 'Cancel_Rate', 'Total_Flights']
        month_cancel['Cancel_Rate'] = month_cancel['Cancel_Rate'] * 100
        month_names = {1: 'January', 7: 'July', 9: 'September'}
        for month, row in month_cancel.iterrows():
            month_name = month_names.get(month, f'Month {month}')
            print(f"  {month_name}: {row['Cancellations']:,.0f} cancellations ({row['Cancel_Rate']:.2f}%)")
    
    # Cancellation reasons
    if 'CANCELLATION_CODE' in combined_df.columns:
        print(f"\nCancellation reasons (overall):")
        cancel_reasons = combined_df[combined_df['CANCELLED'] == 1]['CANCELLATION_CODE'].value_counts()
        reason_map = {'A': 'Carrier', 'B': 'Weather', 'C': 'NAS', 'D': 'Security'}
        for code, count in cancel_reasons.items():
            reason = reason_map.get(code, code)
            pct = count / combined_df['CANCELLED'].sum() * 100
            print(f"  {code} ({reason}): {count:,} ({pct:.1f}%)")
    
    print(f"{'='*60}\n")
    
    return combined_df

# Usage:
filepaths = [
    "/Users/ronny/Desktop/ML projects/Flight Cancels/data/raw/Jan_25.csv",  # Update with your actual filenames
    "/Users/ronny/Desktop/ML projects/Flight Cancels/data/raw/July_25.csv",
    "/Users/ronny/Desktop/ML projects/Flight Cancels/data/raw/SepT_2025.csv"
]

flights_df = load_bts_data_multiple(
    filepaths, 
    output_path="data/processed/flights_combined.csv"
)

✅ Loaded 539,747 flights from Jan_25.csv
✅ Loaded 631,428 flights from July_25.csv
✅ Loaded 562,439 flights from SepT_2025.csv

💾 Saved merged data to: data/processed/flights_combined.csv

COMBINED DATA SUMMARY
Total flights: 1,733,614
Date range: 2025-01-01 00:00:00 to 2025-09-30 00:00:00
Total cancellations: 34,671.0 (2.00%)

Cancellations by month:
  January: 16,312 cancellations (3.02%)
  July: 15,473 cancellations (2.45%)
  September: 2,886 cancellations (0.51%)

Cancellation reasons (overall):
  B (Weather): 25,216 (72.7%)
  A (Carrier): 5,379 (15.5%)
  C (NAS): 4,068 (11.7%)
  D (Security): 8 (0.0%)



In [82]:
# Get airport statistics (origin + destination combined)
origin_counts = flights_df['ORIGIN'].value_counts()
dest_counts = flights_df['DEST'].value_counts()

# Combine and sum (airports appear in both origin and dest)
airport_counts = pd.concat([origin_counts, dest_counts]).groupby(level=0).sum().sort_values(ascending=False)

print("=" * 60)
print("AIRPORT ANALYSIS")
print("=" * 60)
print(f"Total unique airports: {len(airport_counts)}")
print(f"Total flights: {airport_counts.sum():,}")

# Top 50 airports
top_50 = airport_counts.head(50)
print(f"\n📊 Top 50 Airports (by flight volume):")
print(f"   Total flights in top 50: {top_50.sum():,} ({top_50.sum()/airport_counts.sum()*100:.1f}% of all flights)")
print(f"\nTop 50 airports:")
for i, (airport, count) in enumerate(top_50.items(), 1):
    pct = count / airport_counts.sum() * 100
    print(f"   {i:2d}. {airport}: {count:,} flights ({pct:.2f}%)")

# Save top 50
top_50.to_csv("data/processed/top_50_airports.csv", header=['Flight_Count'])
print(f"\n💾 Saved top 50 to: data/processed/top_50_airports.csv")

# All airports (sorted by code)
all_airports = sorted(airport_counts.index.tolist())
print(f"\n📋 All airports ({len(all_airports)} total, sorted by code):")
print(all_airports)

# Save all airports
airport_counts.to_csv("data/processed/origin_airports.csv", header=['Flight_Count'])
print(f"💾 Saved all airports to: data/processed/origin_airports.csv")

# Coverage analysis
print(f"\n📈 Coverage Analysis:")
print(f"   Top 10 airports: {airport_counts.head(10).sum()/airport_counts.sum()*100:.1f}% of flights")
print(f"   Top 25 airports: {airport_counts.head(25).sum()/airport_counts.sum()*100:.1f}% of flights")
print(f"   Top 50 airports: {airport_counts.head(50).sum()/airport_counts.sum()*100:.1f}% of flights")
print(f"   Top 100 airports: {airport_counts.head(100).sum()/airport_counts.sum()*100:.1f}% of flights")

AIRPORT ANALYSIS
Total unique airports: 348
Total flights: 3,467,228

📊 Top 50 Airports (by flight volume):
   Total flights in top 50: 2,714,667 (78.3% of all flights)

Top 50 airports:
    1. ORD: 162,019 flights (4.67%)
    2. DEN: 161,432 flights (4.66%)
    3. DFW: 159,448 flights (4.60%)
    4. ATL: 156,550 flights (4.52%)
    5. CLT: 97,050 flights (2.80%)
    6. LAX: 95,035 flights (2.74%)
    7. PHX: 90,919 flights (2.62%)
    8. LAS: 90,272 flights (2.60%)
    9. SEA: 86,043 flights (2.48%)
   10. MCO: 75,773 flights (2.19%)
   11. DCA: 71,504 flights (2.06%)
   12. SFO: 71,248 flights (2.05%)
   13. BOS: 70,268 flights (2.03%)
   14. LGA: 67,978 flights (1.96%)
   15. EWR: 61,122 flights (1.76%)
   16. DTW: 61,023 flights (1.76%)
   17. IAH: 58,114 flights (1.68%)
   18. SLC: 58,021 flights (1.67%)
   19. MSP: 57,216 flights (1.65%)
   20. MIA: 53,657 flights (1.55%)
   21. JFK: 52,306 flights (1.51%)
   22. BNA: 51,091 flights (1.47%)
   23. BWI: 47,964 flights (1.38%)
   2

## Import and Merge NOAA data


In [83]:
def get_airport_coordinates_geocoding(airports_list):
    """
    Get airport coordinates using Nominatim (OpenStreetMap) - free, no API key.
    """
    import time
    
    coords = []
    failed = []
    
    print(f"🌍 Fetching coordinates for {len(airports_list)} airports...")
    
    for i, airport in enumerate(airports_list, 1):
        try:
            # Try airport code search
            url = "https://nominatim.openstreetmap.org/search"
            params = {
                'q': f"{airport} airport USA",
                'format': 'json',
                'limit': 1
            }
            headers = {'User-Agent': 'Flight-Cancellation-Project-1.0'}  # Required by Nominatim
            
            response = requests.get(url, params=params, headers=headers, timeout=10)
            data = response.json()
            
            if data and len(data) > 0:
                coords.append({
                    'airport_code': airport,
                    'lat': float(data[0]['lat']),
                    'lon': float(data[0]['lon']),
                    'airport_name': data[0].get('display_name', '')
                })
                if i % 10 == 0:  # Progress update every 10 (since only 50 total)
                    print(f"   Progress: {i}/{len(airports_list)} airports")
            else:
                failed.append(airport)
            
            time.sleep(1)  # Rate limiting: 1 request per second (free tier)
            
        except Exception as e:
            print(f"   Error for {airport}: {e}")
            failed.append(airport)
    
    df = pd.DataFrame(coords)
    print(f"\n✅ Found coordinates for {len(df)}/{len(airports_list)} airports")
    
    if failed:
        print(f"⚠️ Failed to find: {len(failed)} airports")
        print(f"   Failed airports: {failed}")
    
    return df

# Load top 50 airports from CSV
top_50_df = pd.read_csv("data/processed/top_50_airports.csv", index_col=0)
top_50_airports = top_50_df.index.tolist()  # Get airport codes (index)

print("=" * 60)
print("FETCHING COORDINATES FOR TOP 50 AIRPORTS")
print("=" * 60)
print(f"📊 Top 50 airports to fetch:")
print(f"   {', '.join(top_50_airports)}")
print(f"\n   Total flights in top 50: {top_50_df['Flight_Count'].sum():,}")
print(f"   Coverage: {top_50_df['Flight_Count'].sum() / (flights_df['ORIGIN'].value_counts().sum() + flights_df['DEST'].value_counts().sum()) * 100:.1f}% of all flights")

# Get coordinates (this will take ~50 seconds for 50 airports)
airport_coords_top50 = get_airport_coordinates_geocoding(top_50_airports)

# Save
airport_coords_top50.to_csv("data/processed/airport_coordinates_top50.csv", index=False)
print(f"\n💾 Saved to: data/processed/airport_coordinates_top50.csv")

# Display results
print(f"\n📋 Coordinates fetched:")
print(airport_coords_top50[['airport_code', 'lat', 'lon']].head(10))

FETCHING COORDINATES FOR TOP 50 AIRPORTS
📊 Top 50 airports to fetch:
   ORD, DEN, DFW, ATL, CLT, LAX, PHX, LAS, SEA, MCO, DCA, SFO, BOS, LGA, EWR, DTW, IAH, SLC, MSP, MIA, JFK, BNA, BWI, PHL, SAN, AUS, FLL, MDW, TPA, DAL, STL, PDX, HNL, SMF, IAD, HOU, RDU, MSY, MCI, SJC, IND, SNA, CMH, PIT, SAT, CLE, OAK, SJU, CVG, RSW

   Total flights in top 50: 2,714,667
   Coverage: 78.3% of all flights
🌍 Fetching coordinates for 50 airports...
   Progress: 10/50 airports
   Progress: 20/50 airports
   Progress: 30/50 airports
   Progress: 40/50 airports
   Progress: 50/50 airports

✅ Found coordinates for 50/50 airports

💾 Saved to: data/processed/airport_coordinates_top50.csv

📋 Coordinates fetched:
  airport_code        lat         lon
0          ORD  41.978252  -87.909235
1          DEN  39.860668 -104.685367
2          DFW  32.896519  -97.046522
3          ATL  33.637401  -84.429816
4          CLT  35.210741  -80.945744
5          LAX  33.942167 -118.421359
6          PHX  33.432849 -112.00679

In [ ]:
def fetch_weather_data_top50(flights_df, airport_coords_df, api_key, output_path=None):
    """
    Fetch weather data for top 50 airports only and merge with flights.
    
    Args:
        flights_df: DataFrame with flight data (must have ORIGIN, DEST, FL_DATE)
        airport_coords_df: DataFrame with top 50 airport coordinates (airport_code, lat, lon)
        api_key: Visual Crossing API key
        output_path: Optional path to save merged data
    
    Returns:
        Merged DataFrame with weather data
    """
    import time
    
    print("=" * 60)
    print("FETCHING WEATHER DATA (TOP 50 AIRPORTS ONLY)")
    print("=" * 60)
    
    # Get list of top 50 airports from coordinates
    top_50_airports = set(airport_coords_df['airport_code'].tolist())
    print(f"\n📊 Top 50 airports: {len(top_50_airports)} airports")
    
    # Filter flights to only include top 50 airports
    print("\n🔍 Filtering flights to top 50 airports...")
    flights_filtered = flights_df[
        flights_df['ORIGIN'].isin(top_50_airports) & 
        flights_df['DEST'].isin(top_50_airports)
    ].copy()
    
    original_count = len(flights_df)
    filtered_count = len(flights_filtered)
    coverage = filtered_count / original_count * 100
    
    print(f"   Original flights: {original_count:,}")
    print(f"   Filtered flights: {filtered_count:,} ({coverage:.1f}% of total)")
    
    if filtered_count == 0:
        print("❌ No flights found with top 50 airports!")
        return flights_df
    
    # Create airport coordinate lookup
    coord_lookup = airport_coords_df.set_index('airport_code')[['lat', 'lon']].to_dict('index')
    
    # Get unique airport-date combinations (for both origin and dest)
    print("\n📊 Analyzing flight data...")
    
    # Origin airports
    origin_dates = flights_filtered[['ORIGIN', 'FL_DATE']].drop_duplicates()
    origin_dates.columns = ['airport', 'date']
    
    # Destination airports
    dest_dates = flights_filtered[['DEST', 'FL_DATE']].drop_duplicates()
    dest_dates.columns = ['airport', 'date']
    
    # Combine and get unique combinations
    all_airport_dates = pd.concat([origin_dates, dest_dates]).drop_duplicates()
    all_airport_dates['date'] = pd.to_datetime(all_airport_dates['date']).dt.date
    
    print(f"   Unique airport-date combinations: {len(all_airport_dates):,}")
    print(f"   (Much faster than {len(flights_df[['ORIGIN', 'FL_DATE']].drop_duplicates()) + len(flights_df[['DEST', 'FL_DATE']].drop_duplicates()):,} for all airports!)")
    
    # Fetch weather data
    weather_data = []
    failed = []
    consecutive_429s = 0
    
    for idx, row in all_airport_dates.iterrows():
        airport = row['airport']
        date = row['date']
        
        # Get coordinates (should always exist since we filtered)
        if airport not in coord_lookup:
            failed.append((airport, date, "No coordinates"))
            continue
        
        lat = coord_lookup[airport]['lat']
        lon = coord_lookup[airport]['lon']
        
        retries = 0
        max_retries = 3
        
        while retries < max_retries:
            try:
                # Visual Crossing API call
                url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
                params = {
                    'location': f"{lat},{lon}",
                    'date': date.strftime('%Y-%m-%d'),
                    'key': api_key,
                    'include': 'days',
                    'elements': 'datetime,temp,precip,windspeed,visibility,snow,conditions'
                }
                
                response = requests.get(url, params=params, headers={'User-Agent': 'Flight-Cancellation-Project'}, timeout=30)
                
                if response.status_code == 200:
                    data = response.json()
                    
                    if 'days' in data and len(data['days']) > 0:
                        day_data = data['days'][0]
                        weather_data.append({
                            'airport': airport,
                            'date': date,
                            'temp_max': day_data.get('tempmax'),
                            'temp_min': day_data.get('tempmin'),
                            'temp_avg': day_data.get('temp'),
                            'precip': day_data.get('precip', 0),
                            'snow': day_data.get('snow', 0),
                            'windspeed': day_data.get('windspeed'),
                            'visibility': day_data.get('visibility'),
                            'conditions': day_data.get('conditions', '')
                        })
                        consecutive_429s = 0  # Reset on success
                    
                    break  # Success, exit retry loop
                
                elif response.status_code == 429:
                    consecutive_429s += 1
                    wait_time = min(60 * consecutive_429s, 300)  # Max 5 min wait
                    print(f"   ⚠️ Rate limit (429). Waiting {wait_time} seconds... (attempt {retries+1}/{max_retries})")
                    time.sleep(wait_time)
                    retries += 1
                    
                    if retries >= max_retries:
                        print(f"   ❌ Max retries reached for {airport} on {date}")
                        failed.append((airport, date, "Rate limit exceeded"))
                        break
                
                elif response.status_code == 401:
                    print("❌ API key invalid!")
                    return None
                
                else:
                    failed.append((airport, date, f"Status {response.status_code}"))
                    break
                    
            except Exception as e:
                if "429" in str(e):
                    consecutive_429s += 1
                    wait_time = min(60 * consecutive_429s, 300)
                    print(f"   ⚠️ Rate limit error. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"   ⚠️ Error for {airport} on {date}: {e}")
                    failed.append((airport, date, str(e)))
                    break
        
        # Progress update
        if len(weather_data) % 50 == 0:
            print(f"   Progress: {len(weather_data)}/{len(all_airport_dates)} fetched")
        
        # Rate limiting: 2 seconds between requests (more conservative)
        time.sleep(2)
    
    # Create weather DataFrame
    weather_df = pd.DataFrame(weather_data)
    
    if len(weather_df) > 0:
        weather_df['date'] = pd.to_datetime(weather_df['date'])
        print(f"\n✅ Fetched weather for {len(weather_df):,} airport-date combinations")
    else:
        print("\n❌ No weather data fetched!")
        return flights_filtered
    
    if failed:
        print(f"⚠️ Failed to fetch: {len(failed)} combinations")
        if len(failed) <= 10:
            print(f"   Failed: {failed}")
    
    # Merge with flight data (use filtered flights)
    print("\n🔗 Merging weather data with flights...")
    
    # Merge origin weather
    flights_merged = flights_filtered.merge(
        weather_df,
        left_on=['ORIGIN', 'FL_DATE'],
        right_on=['airport', 'date'],
        how='left',
        suffixes=('', '_origin')
    )
    
    # Rename origin weather columns
    origin_cols = ['temp_max', 'temp_min', 'temp_avg', 'precip', 'snow', 'windspeed', 'visibility', 'conditions']
    rename_dict = {col: f'{col}_origin' for col in origin_cols if col in flights_merged.columns}
    flights_merged = flights_merged.rename(columns=rename_dict)
    
    # Drop merge helper columns
    flights_merged = flights_merged.drop(columns=['airport', 'date'], errors='ignore')
    
    # Merge destination weather
    flights_merged = flights_merged.merge(
        weather_df,
        left_on=['DEST', 'FL_DATE'],
        right_on=['airport', 'date'],
        how='left',
        suffixes=('', '_dest')
    )
    
    # Rename destination weather columns
    rename_dict = {col: f'{col}_dest' for col in origin_cols if col in flights_merged.columns}
    flights_merged = flights_merged.rename(columns=rename_dict)
    
    # Drop merge helper columns
    flights_merged = flights_merged.drop(columns=['airport', 'date'], errors='ignore')
    
    # Summary
    print(f"\n✅ Merged weather data")
    print(f"   Total flights (filtered): {len(flights_merged):,}")
    print(f"   Flights with origin weather: {flights_merged['temp_avg_origin'].notna().sum():,} ({flights_merged['temp_avg_origin'].notna().mean()*100:.1f}%)")
    print(f"   Flights with dest weather: {flights_merged['temp_avg_dest'].notna().sum():,} ({flights_merged['temp_avg_dest'].notna().mean()*100:.1f}%)")
    
    # Save if output path provided
    if output_path:
        flights_merged.to_csv(output_path, index=False)
        print(f"\n💾 Saved merged data to: {output_path}")
    
    print("=" * 60)
    
    return flights_merged

# Usage:
# 1. Load your data
airport_coords_top50 = pd.read_csv("data/processed/airport_coordinates_top50.csv")
flights_df = pd.read_csv("data/processed/flights_combined.csv")
flights_df['FL_DATE'] = pd.to_datetime(flights_df['FL_DATE'])

# 2. Set your Visual Crossing API key
api_key = "G2SRNMYAF44NW4YYGMFXKK9UV"  # Replace with your actual key

# 3. Fetch and merge weather data (only for top 50 airports)
flights_with_weather = fetch_weather_data_top50(
    flights_df,
    airport_coords_top50,
    api_key,
    output_path="data/processed/flights_with_weather_top50.csv"
)

In [ ]:
def fetch_weather_openmeteo_complete(flights_df, airport_coords_df, output_path=None):
    """
    Complete weather fetching using Open-Meteo (FREE, no API key).
    """
    import time
    
    print("=" * 60)
    print("FETCHING WEATHER FROM OPEN-METEO (FREE)")
    print("=" * 60)
    
    # Filter to top 50 airports
    top_50_airports = set(airport_coords_df['airport_code'].tolist())
    flights_filtered = flights_df[
        flights_df['ORIGIN'].isin(top_50_airports) & 
        flights_df['DEST'].isin(top_50_airports)
    ].copy()
    
    # Get unique combinations
    origin_dates = flights_filtered[['ORIGIN', 'FL_DATE']].drop_duplicates()
    dest_dates = flights_filtered[['DEST', 'FL_DATE']].drop_duplicates()
    all_airport_dates = pd.concat([
        origin_dates.rename(columns={'ORIGIN': 'airport', 'FL_DATE': 'date'}),
        dest_dates.rename(columns={'DEST': 'airport', 'FL_DATE': 'date'})
    ]).drop_duplicates()
    
    coord_lookup = airport_coords_df.set_index('airport_code')[['lat', 'lon']].to_dict('index')
    
    print(f"📊 Fetching weather for {len(all_airport_dates):,} airport-date combinations")
    
    weather_data = []
    
    for idx, row in all_airport_dates.iterrows():
        airport = row['airport']
        date = pd.to_datetime(row['date'])
        
        if airport not in coord_lookup:
            continue
        
        lat = coord_lookup[airport]['lat']
        lon = coord_lookup[airport]['lon']
        
        try:
            url = "https://archive-api.open-meteo.com/v1/archive"
            params = {
                'latitude': lat,
                'longitude': lon,
                'start_date': date.strftime('%Y-%m-%d'),
                'end_date': date.strftime('%Y-%m-%d'),
                'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max,windgusts_10m_max',
                'timezone': 'auto'
            }
            
            response = requests.get(url, params=params, timeout=30)
            
            if response.status_code == 200:
                data = response.json()
                if 'daily' in data and data['daily']:
                    daily = data['daily']
                    temp_max = daily.get('temperature_2m_max', [None])[0]
                    temp_min = daily.get('temperature_2m_min', [None])[0]
                    
                    weather_data.append({
                        'airport': airport,
                        'date': date.date(),
                        'temp_max': temp_max,
                        'temp_min': temp_min,
                        'temp_avg': (temp_max + temp_min) / 2 if temp_max and temp_min else None,
                        'precip': daily.get('precipitation_sum', [0])[0] or 0,
                        'windspeed': daily.get('windspeed_10m_max', [None])[0],
                        'windgust': daily.get('windgusts_10m_max', [None])[0],
                    })
            
            if len(weather_data) % 50 == 0:
                print(f"   Progress: {len(weather_data)}/{len(all_airport_dates)}")
            
            time.sleep(0.5)  # 2 requests/sec
            
        except Exception as e:
            print(f"   ⚠️ Error for {airport}: {e}")
    
    weather_df = pd.DataFrame(weather_data)
    if len(weather_df) > 0:
        weather_df['date'] = pd.to_datetime(weather_df['date'])
    
    # Merge with flights (same as before)
    # ... (merge logic)
    
    return flights_merged

# Usage (NO API KEY NEEDED!):
flights_with_weather = fetch_weather_openmeteo_complete(
    flights_df,
    airport_coords_top50,
    output_path="data/processed/flights_with_weather_openmeteo.csv"
)

FETCHING WEATHER FROM OPEN-METEO (FREE)
📊 Fetching weather for 4,600 airport-date combinations


## Phase 4: Data Preprocessing & Feature Engineering

**Goal**: Clean data, create features (time-based, weather, airport, airline, route)


## Phase 5: Baseline Models

**Goal**: Build simple prediction models (Logistic Regression, XGBoost) to establish baseline performance


In [ ]:
# Baseline models


## Phase 6: Time Series Models

**Goal**: Use transformers (PatchTST/TST) to capture temporal patterns in flight cancellations


In [ ]:
# Time series transformer models


## Phase 7: Causal Inference Setup

**Goal**: Identify causal relationships - separate weather effects from operational effects


In [ ]:
# Causal inference (difference-in-differences, propensity score matching)


## Phase 8: Attribution Model

**Goal**: Attribute each cancellation to specific causes (weather vs. mechanical vs. crew vs. other)


In [ ]:
# Attribution using Shapley values, counterfactual analysis


## Phase 9: Integration & Evaluation

**Goal**: Combine prediction + attribution, evaluate end-to-end system


In [ ]:
# End-to-end evaluation


## Phase 10: Results & Insights

**Goal**: Summarize findings, visualize results, document key insights


In [ ]:
# Final results, visualizations, and conclusions


## Phase 3: Exploratory Data Analysis (EDA)

**Goal**: Understand data distributions, cancellation patterns, correlations
